# XGMIX notebook

This seems to work. One caveat is that it gets stuck in an infinte loop for the fist time you run after making changes. So just stop the kernel and re-run the cell.

In [1]:
import numpy as np
import os

In [2]:
%load_ext autoreload
%autoreload 2
from preprocess import data_process,simulate_missing_values
from XGMIX import *

## Main arguments

In [3]:
# args = {'train_data': "/home/arvindsk/xgmix_new/expts/xgmix_global_1/train.txt", 
#         "val_data" : "/home/arvindsk/xgmix_new/expts/xgmix_global_1/val.txt", 
#         "save": None,
#         "window_size": 1000,
#         "smooth_size": 75,
#         "missing":0.0}

In [29]:
args = {'train_data': "/home/arvindsk/xgmix_new/expts/xgmix_global_22/train.txt", 
        "val_data" : "/home/arvindsk/xgmix_new/expts/xgmix_global_22/val.txt", 
        "save": "/home/arvindsk/xgmix_new/expts/xgmix_global_22/",
        "window_size": 1000,
        "smooth_size": 75,
        "missing":0.0}

## Pre-processing

In [30]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
args = Struct(**args)

win = args.window_size
training, training_labels, validation, validation_labels = data_process(args)
chmlen = training.shape[-1]
sws = args.smooth_size if args.smooth_size%2 else args.smooth_size-1
num_anc = len(np.unique(training_labels))

Loading data...
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_2//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_4//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_6//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_2//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_4//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_6//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_2//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_4//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_6//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_v

In [15]:
if args.missing != 0:
    training = simulate_missing_values(training,args.missing)
    validation = simulate_missing_values(validation,args.missing)

In [ ]:
try:
    os.mkdir(args.save)
except:
    print("Directory exists")

In [ ]:
xgm = XGMIX(chmlen,win,sws,num_anc,args.save,cores=16)

In [ ]:
xgm.train(training,training_labels,validation, validation_labels, smoothlite=10000)

In [ ]:
ypreds = predict(validation,args.save)

In [ ]:
np.mean(ypreds == validation_labels.numpy())

In [ ]:
sklearn.metrics.confusion_matrix(ypreds.ravel(), validation_labels.numpy().ravel())